# Kong: Performance Evaluation

In [11]:
import matplotlib.pyplot as plt
import numpy as np
import os

## Benchmarks

In [12]:
# Export the environment variable needed by Kong (need to be modified for your configuration!)
os.environ["PNML2NUPN"] = "~/pnml2nupn-3.2.0.jar"

In [13]:
MODELS_URL = "https://mcc.lip6.fr/archives/"
MODELS_DIR = "models/"

os.system("mkdir {}".format(MODELS_DIR))
os.system("wget -r --no-parent -A '*pnml.tar.gz' {} -P {}".format(MODELS_URL, MODELS_DIR)) 
os.system("ls {}*.tar.gz | xargs -n1 tar -xvf".format(MODELS_DIR))
os.system("rm {}*.tar.gz".format(MODELS_DIR)) 


256